## Setup

In [1]:
GLOBAL_SEED = 42

import os
os.environ['PYTHONHASHSEED'] = str(GLOBAL_SEED)
import sys

import gc
from tqdm import tqdm
import datetime
import pickle
import random as rnd
from glob import glob
import pandas as pd
import numpy as np
from numpy import random as np_rnd
import warnings
from math import ceil

import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import rcParams
from itertools import combinations

from scipy.stats import f_oneway
from scipy.stats import pearsonr

# display setting
warnings.filterwarnings(action='ignore')
rcParams['axes.unicode_minus'] = False

In [2]:
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    # python random
    rnd.seed(seed)
    # numpy random
    np_rnd.seed(seed)
    # tf random
    try:
        tf_rnd.set_seed(seed)
    except:
        pass
    # RAPIDS random
    try:
        cupy.random.seed(seed)
    except:
        pass
    # pytorch random
    try:
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
    except:
        pass

def create_get_ts(ts):
    return int((ts.replace(tzinfo=CFG.tz) - CFG.ts_zero).total_seconds())

def pickleIO(obj, src, op="w"):
    if op=="w":
        with open(src, op + "b") as f:
            pickle.dump(obj, f)
    elif op=="r":
        with open(src, op + "b") as f:
            tmp = pickle.load(f)
        return tmp
    else:
        print("unknown operation")
        return obj

def week_of_month(dt):
    """ 
        Returns the week of the month for the specified date.
    """
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + (1 + first_day.weekday()) % 7
    return int(ceil(adjusted_dom/7.0))

In [3]:
class CFG:
    debug = False
    product_mapper = {
        "A": ["T010305", "T010306", "T050304", "T050307"],
        "O": ["T100304", "T100306"],
        "T": ["T100304", "T100306"],
    }
    line_mapper = {
        "T010305": "A", "T010306": "A", "T050304": "A", "T050307": "A",
        "T100304": "O_T", "T100306": "O_T",
    }

## Loading Data

In [4]:
# information Provided by Dacon

# PRODUCT_ID : 제품의 고유 ID
# Y_Class : 제품 품질 상태(Target) 
# 0 : 적정 기준 미달 (부적합)
# 1 : 적합
# 2 : 적정 기준 초과 (부적합)
# Y_Quality : 제품 품질 관련 정량적 수치
# TIMESTAMP : 제품이 공정에 들어간 시각
# LINE : 제품이 들어간 공정 LINE 종류 ('T050304', 'T050307', 'T100304', 'T100306', 'T010306', 'T010305' 존재)
# PRODUCT_CODE : 제품의 CODE 번호 ('A_31', 'T_31', 'O_31' 존재)
# X_1 ~ X_2875 : 공정 과정에서 추출되어 비식별화된 변수

In [5]:
df_full = pd.read_csv("C:/Users/flash/PycharmProjects/pythonProject/projects/dacon_lgaimers2/datasets/train.csv")

In [6]:
df_full.columns = df_full.columns.str.lower()
df_full["timestamp"] = pd.to_datetime(df_full["timestamp"])

In [7]:
# # time feature engineernig
# df_full["month"] = df_full["timestamp"].dt.month
# df_full["day"] = df_full["timestamp"].dt.day
# df_full["weekday"] = df_full["timestamp"].dt.weekday
# df_full["week_of_month"] = df_full["timestamp"].apply(week_of_month)
# df_full["hour"] = df_full["timestamp"].dt.hour
# df_full["office_hour"] = df_full["hour"].apply(lambda x: 1 if ((x >= 9) & (x < 18)) else 0)
# df_full["sec_in_day"] = (df_full["timestamp"] - df_full["timestamp"].dt.normalize()).dt.total_seconds() / 3600
# df_full["sin_in_day"] = np.sin(2 * np.pi * df_full["sec_in_day"].values)
# df_full["cos_in_day"] = np.cos(2 * np.pi * df_full["sec_in_day"].values)

## Null Value Analysis

### Create dataframe by lines & products

In [8]:
# normalized 된 null 값 count dataframe 생성
# 1이면 해당 그룹은 모두 해당 feature가 nan 값임을 의미
df_tmp = df_full.filter(regex="line|product_code|x_*").groupby(["line", "product_code"]).agg(lambda x: x.isnull().sum() / len(x))
df_tmp

x_1  x_2  x_3  x_4  x_5  x_6  x_7  x_8  x_9  x_10  ...  \
line    product_code                                                     ...   
T010305 A_31          1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   1.0  ...   
T010306 A_31          1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   1.0  ...   
T050304 A_31          1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   1.0  ...   
T050307 A_31          1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   1.0  ...   
T100304 O_31          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ...   
        T_31          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ...   
T100306 O_31          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ...   
        T_31          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ...   

                        x_2866    x_2867    x_2868    x_2869    x_2870  \
line    product_code                                                     
T010305 A_31          1.000000  1.000000  1.000000  1.000000  1.000000   
T010306 A_31          1.000000  1.000000  1.000000  1.000000  1.000000   
T050304 A_31          0.141026  0.141026  0.141026  0.141026  0.141026   
T050307 A_31          0.214286  0.214286  0.214286  0.214286  0.214286   
T100304 O_31          1.000000  1.000000  1.000000  1.000000  1.000000   
        T_31          1.000000  1.000000  1.000000  1.000000  1.000000   
T100306 O_31          1.000000  1.000000  1.000000  1.000000  1.000000   
        T_31          1.000000  1.000000  1.000000  1.000000  1.000000   

                        x_2871  x_2872  x_2873  x_2874  x_2875  
line    product_code                                            
T010305 A_31          1.000000     1.0     1.0     1.0     1.0  
T010306 A_31          1.000000     1.0     1.0     1.0     1.0  
T050304 A_31          0.141026     1.0     1.0     1.0     1.0  
T050307 A_31          0.238095     1.0     1.0     1.0     1.0  
T100304 O_31          1.000000     1.0     1.0     1.0     1.0  
        T_31          1.000000     1.0     1.0     1.0     1.0  
T100306 O_31          1.000000     1.0     1.0     1.0     1.0  
        T_31          1.000000     1.0     1.0     1.0     1.0  

[8 rows x 2875 columns]

### line 이름 앞 3자리에 따라 feature가 다른 것을 알고 line 별로만 분석

In [9]:
# line 별로 count를 다시 계산 후
# 1이 아닌 feature 만 선택, 즉 모든 값이 nan은 아닌 feature

df_line = df_tmp.groupby("line").mean()

line_no_null = dict.fromkeys(df_line.index)
for i in df_line.index:
    line_no_null[i] = list(df_line.columns[df_line.loc[i] != 1.0])

In [10]:
# line의 이름 앞 3자리에 따라 null이 아닌 feature 갯수가 유사함을 알 수 있음
# 이는 우연일 수 있어 심층 분석이 필요
for k, v in line_no_null.items():
    print("Line -> ", k)
    print("Non-Null Count :", len(v))
    print("Non-Null Count Ratio :",np.round(len(v) / 2875, 3), "\n")
line_no_null = pd.Series(line_no_null)

Line ->  T010305
Non-Null Count : 886
Non-Null Count Ratio : 0.308 

Line ->  T010306
Non-Null Count : 886
Non-Null Count Ratio : 0.308 

Line ->  T050304
Non-Null Count : 1968
Non-Null Count Ratio : 0.685 

Line ->  T050307
Non-Null Count : 1975
Non-Null Count Ratio : 0.687 

Line ->  T100304
Non-Null Count : 671
Non-Null Count Ratio : 0.233 

Line ->  T100306
Non-Null Count : 671
Non-Null Count Ratio : 0.233 



In [11]:
line_no_null

T010305    [x_246, x_247, x_248, x_249, x_250, x_251, x_2...
T010306    [x_246, x_247, x_248, x_249, x_250, x_251, x_2...
T050304    [x_128, x_129, x_132, x_133, x_134, x_135, x_1...
T050307    [x_130, x_131, x_132, x_133, x_134, x_135, x_1...
T100304    [x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, ...
T100306    [x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, ...
dtype: object

### 컬럼 교차 정보 파악을 위한 dataframe 생성 (공정 6개 조합별 - 6C2)

In [12]:
tmp = {
    "line_1": [],
    "line_2": [],
    "교차컬럼갯수": [],
    "교차하지않는_컬럼갯수": [],
    "교차컬럼갯수/line1컬럼갯수": [],
    "교차컬럼갯수/line2컬럼갯수": [],
    "교차컬럼갯수/합집합컬럼갯수": [],
    "교차하지않는_컬럼갯수/합집합컬럼갯수": [],
}

for i, j in combinations(line_no_null.index, 2):
    tmp["line_1"].append(i)
    tmp["line_2"].append(j)
    tmp["교차컬럼갯수"].append(len(np.intersect1d(line_no_null.loc[i], line_no_null.loc[j])))
    tmp["교차하지않는_컬럼갯수"].append(len(np.setxor1d(line_no_null.loc[i], line_no_null.loc[j])))
    tmp["교차컬럼갯수/line1컬럼갯수"].append(tmp["교차컬럼갯수"][-1] / len(line_no_null.loc[i]))
    tmp["교차컬럼갯수/line2컬럼갯수"].append(tmp["교차컬럼갯수"][-1] / len(line_no_null.loc[j]))
    tmp["교차컬럼갯수/합집합컬럼갯수"].append(tmp["교차컬럼갯수"][-1] / len(np.union1d(line_no_null.loc[i], line_no_null.loc[j])))
    tmp["교차하지않는_컬럼갯수/합집합컬럼갯수"].append(tmp["교차하지않는_컬럼갯수"][-1] / len(np.union1d(line_no_null.loc[i], line_no_null.loc[j])))
    
tmp = pd.DataFrame(tmp)
tmp["line_1_제품"] = tmp["line_1"].map(CFG.line_mapper)
tmp["line_2_제품"] = tmp["line_2"].map(CFG.line_mapper)
tmp= tmp[list(tmp.columns[:2]) + list(tmp.columns[-2:]) + list(tmp.columns[2:-2])]

In [13]:
# line 조합 별 feature 교차 갯수 분석 데이터프레임
tmp

,line_1,line_2,line_1_제품,line_2_제품,교차컬럼갯수,교차하지않는_컬럼갯수,교차컬럼갯수/line1컬럼갯수,교차컬럼갯수/line2컬럼갯수,교차컬럼갯수/합집합컬럼갯수,교차하지않는_컬럼갯수/합집합컬럼갯수
0,T010305,T010306,A,A,878,16,0.990971,0.990971,0.982103,0.017897
1,T010305,T050304,A,A,765,1324,0.863431,0.388720,0.366204,0.633796
2,T010305,T050307,A,A,760,1341,0.857788,0.384810,0.361733,0.638267
3,T010305,T100304,A,O_T,0,1557,0.000000,0.000000,0.000000,1.000000
4,T010305,T100306,A,O_T,0,1557,0.000000,0.000000,0.000000,1.000000
5,T010306,T050304,A,A,765,1324,0.863431,0.388720,0.366204,0.633796
6,T010306,T050307,A,A,760,1341,0.857788,0.384810,0.361733,0.638267
7,T010306,T100304,A,O_T,0,1557,0.000000,0.000000,0.000000,1.000000
8,T010306,T100306,A,O_T,0,1557,0.000000,0.000000,0.000000,1.000000
9,T050304,T050307,A,A,1956,31,0.993902,0.990380,0.984399,0.015601


In [14]:
# 교차컬럼이 없는 line 조합은 drop
tmp[tmp["교차컬럼갯수"] != 0]

,line_1,line_2,line_1_제품,line_2_제품,교차컬럼갯수,교차하지않는_컬럼갯수,교차컬럼갯수/line1컬럼갯수,교차컬럼갯수/line2컬럼갯수,교차컬럼갯수/합집합컬럼갯수,교차하지않는_컬럼갯수/합집합컬럼갯수
0,T010305,T010306,A,A,878,16,0.990971,0.990971,0.982103,0.017897
1,T010305,T050304,A,A,765,1324,0.863431,0.388720,0.366204,0.633796
2,T010305,T050307,A,A,760,1341,0.857788,0.384810,0.361733,0.638267
5,T010306,T050304,A,A,765,1324,0.863431,0.388720,0.366204,0.633796
6,T010306,T050307,A,A,760,1341,0.857788,0.384810,0.361733,0.638267
9,T050304,T050307,A,A,1956,31,0.993902,0.990380,0.984399,0.015601
14,T100304,T100306,O_T,O_T,665,12,0.991058,0.991058,0.982275,0.017725


### Summary
* T010, T050, T100 세 line 그룹별로 존재하는 feature가 다르다. (LG 측에서 공식적으로 내놓지는 않았으나, 잠정적으로 확실해 보임)
* 위 사실을 뒷바침하는 증거가 제품 A를 생산하는 T010, T050 공정은 일부 겹치는 feature가 있으나, 제품 O & T 를 생산하는 T100 공정과는 겹치는 feature가 하나도 없다.

In [16]:
tmp[tmp["교차컬럼갯수"] != 0].to_csv("kyj_lgimers_eda2.csv", encoding="cp949", index=False)